In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


%run nb_init.py
# This imports leg_joint as lj


/home/guillaume/anaconda/envs/python3/lib/python3.4/site-packages/graph_tool/draw/cairo_draw.py:1458: RuntimeWarning: Error importing Gtk module: No module named 'gi'; GTK+ drawing will not work.
  warnings.warn(msg, RuntimeWarning)
2015-09-22 09:43:30,918 -leg_joint -INFO -successfully imported leg_joint


In [2]:
lj_eptm = lj.Epithelium(lj.data.before_apoptosis_xml(), save_dir='/tmp')

2015-09-22 09:43:34,617 -leg_joint.epithelium -INFO -Instanciating epithelium 0
2015-09-22 09:43:36,304 -leg_joint.epithelium -INFO -Initial cells
2015-09-22 09:43:36,305 -leg_joint.epithelium -INFO -Initial junctions
2015-09-22 09:43:43,061 -leg_joint.epithelium -INFO -Update geometry


In [3]:
import numpy as np
import pandas as pd

import matplotlib.pylab as plt
%matplotlib inline

import tyssue as ty
from tyssue.core import Epithelium, Cell, JunctionEdge, JunctionVertex
from tyssue.core.generation import jv_data, cell_data, je_data
from tyssue.geometry import sheet_geometry as geom

In [4]:
graph = lj_eptm.graph
print('****** Vertex ********')
print(*graph.vertex_properties.keys(), sep='\n')
print('****** Edge ********')

print(*graph.edge_properties.keys(), sep='\n')

      

****** Vertex ********
contractile_grad
grad_wy
vol_elasticities
areas
is_cell_vert
zeds
vols
contractilities
is_alive
is_active_vert
radial_tensions
volume_grad_radial
grad_sigma
ixs
elastic_grad
volume_grad_cell
prefered_vol
sigmas
volume_grad_apical
is_local_vert
rhos
ages
wys
grad_radial
thetas
grad_ix
volume_grad
grad_zed
perimeters
****** Edge ********
is_local_edge
u_dixs
dixs
u_dsigmas
dwys
dthetas
u_dzeds
line_tensions
at_boundary
drhos
is_junction_edge
u_dwys
dzeds
u_drhos
edge_lengths
is_new_edge
dsigmas
is_active_edge
is_ctoj_edge


In [5]:
print('CELLS')
print(*cell_data.keys(), sep='\t')

print('JV')
print(*jv_data.keys(), sep='\t')

print('JE')
print(*je_data.keys(), sep='\t')

CELLS
z	perimeter	area	x	y	num_sides	is_alive
JV
is_active	z	y	x
JE
nx	ny	length	dz	dy	nz	dx


In [6]:
jv_import = {'ixs': 'x', 'wys': 'y', 'zeds': 'z',
             'is_active_vert': 'is_active'}

cell_import = {'ixs': 'x', 'wys': 'y', 'zeds': 'z',
               'areas': 'area', 'perimeters': 'perimeter',
               'is_alive': 'is_alive'}

je_import = {'dixs': 'dx', 'dwys': 'dy', 'dzeds': 'dz',
             'edge_lengths': 'length'}

In [7]:
def get_faces(graph, as_array=True):
    '''
    Retrieves all the triangular subgraphs of the form
       1 -- > 2
        ^   ^
         \ /
          0
    In our context, vertex 0 always corresponds to a cell
    and vertices 1 and 2 to junction vertices

    Parameters
    ----------
    graph : a :class:`GraphTool` graph instance
    as_array: bool, optional, default `True`
      if `True`, the output of `subraph_isomorphism` is converted
      to a (N, 3) ndarray.

    Returns
    -------
    triangles:  list of gt.PropertyMaps or (N, 3) ndarray
      each line corresponds to a triplet (cell, jv0, jv1)
      where cell, jv0 and jv1 are indices of the input graph.
    '''
    tri_graph = gt.Graph()
    ## the vertices
    verts = tri_graph.add_vertex(3)
    ## edges
    tri_graph.add_edge_list([(0, 1), (0, 2), (1, 2)])
    _triangles = gt.subgraph_isomorphism(tri_graph, graph)
    if not as_array:
        return tri_graph, _triangles
    triangles = np.array([tri.a for tri in _triangles], dtype=np.int)
    return triangles

In [8]:
triangles = get_faces(graph)

The new base triangle is oriented differently, as the edge is splited.

We now have:

```
       1 -- > 2             2 <-- 1  
        ^   /                ^   /
         \ v     and          \ v  
          0                    0
```



In [9]:
pos = gt.group_vector_property([lj_eptm.ixs, lj_eptm.wys, lj_eptm.zeds])
deltas = gt.group_vector_property([graph.ep['dixs'], graph.ep['dwys'],  graph.ep['dzeds']])

In [10]:
%pdb

Automatic pdb calling has been turned ON


In [11]:
oriented = []

for (cell, jv1, jv2) in triangles:
    
    normal = np.cross(deltas[graph.edge(cell, jv1)],
                      deltas[graph.edge(cell, jv2)])
    orientation = np.sign(np.sum(pos[graph.vertex(cell)] * normal))
    if orientation > 0:
        oriented.append([jv1, jv2, cell])
    else:
        oriented.append([jv2, jv1, cell])
    
    

In [12]:
graph.set_vertex_filter(lj_eptm.is_cell_vert)
cells_idx_in = graph.vertex_index.copy()
cells_idx_in = cells_idx_in.fa

graph.set_vertex_filter(lj_eptm.is_cell_vert, True)
jv_idx_in = graph.vertex_index.copy()
jv_idx_in = jv_idx_in.fa
graph.set_vertex_filter(None)



In [13]:
cell_df = pd.DataFrame(index=pd.Index(range(cells_idx_in.shape[0]), name='cell'), 
                       columns=cell_import.values())
cell_df['old_idx'] = cells_idx_in

for data_in, data_out in cell_import.items():
    cell_df[data_out] = graph.vp[data_in].a[cells_idx_in]

jv_df = pd.DataFrame(index=pd.Index(range(jv_idx_in.shape[0]), name='jv'), 
                       columns=jv_import.values())
jv_df['old_idx'] = jv_idx_in
    
    
for data_in, data_out in jv_import.items():
    jv_df[data_out] = graph.vp[data_in].a[jv_idx_in]



In [14]:
re_indexed = []
for jv0, jv1, cell in oriented:
    srce = np.where(jv_df.old_idx==jv0)[0][0]
    trgt = np.where(jv_df.old_idx==jv1)[0][0]
    r_cell = np.where(cell_df.old_idx==cell)[0][0]
    re_indexed.append((srce, trgt, r_cell))

In [15]:
je_idx = pd.MultiIndex.from_tuples(re_indexed, names=['srce', 'trgt', 'cell'])
je_df = pd.DataFrame(index=je_idx, columns=je_import.values())
je_df['old_jv0'] = triangles[:, 1]
je_df['old_jv1'] = triangles[:, 2]
je_df['old_cell'] = triangles[:, 0]


eps = gt.group_vector_property([graph.ep[col] for col in je_import.keys()])


#for halfedge in je_idx:
    
#    je_df[[v for v in je_import.values()]].loc[halfedge] = eps[graph.edge(je_df.old_jv0.loc[halfedge],
#                                                             je_df.old_jv1.loc[halfedge])]

In [16]:
je_df

,,,length,dz,dy,dx,old_jv0,old_jv1,old_cell
srce,trgt,cell,,,,,,,
1,0,0,NaN,NaN,NaN,NaN,928,929,0
2,1,0,NaN,NaN,NaN,NaN,929,930,0
0,1,1,NaN,NaN,NaN,NaN,928,929,1
3,0,1,NaN,NaN,NaN,NaN,928,931,1
1,6,1,NaN,NaN,NaN,NaN,929,934,1
4,3,1,NaN,NaN,NaN,NaN,931,932,1
5,4,1,NaN,NaN,NaN,NaN,932,933,1
6,5,1,NaN,NaN,NaN,NaN,933,934,1
7,2398,2,NaN,NaN,NaN,NaN,3630,935,2


In [17]:
with pd.get_store('before_apoptosis.hf5') as store:
    store.put('cell_df', cell_df, format='t', dropna=False)
    store.put('jv_df', jv_df, format='t', dropna=False)
    store.put('je_df', je_df, format='t', dropna=False)